如同前几篇的可视化，这里采用的也是jupyter notebook来进行曲线绘制。

In [ ]:
# 加载必要的库
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys, os, caffe
# 设置当前目录
caffe_root = '/home/ubuntu/Downloads/caffe-master/'
sys.path.insert(0, caffe_root+'python')
os.chdir(caffe_root)

设置求解器，和C++/caffe一样，需要一个solver配置文件。

In [ ]:
# set the solver prototxt
caffe.set_mode_cpu()
solver = caffe.SGDSolver('examples/cifar10/cifar10_quick_solver.prototxt')

如果不需要绘制曲线，只需要训练出一个caffemodel，直接调用solver.solve()就可以了。如果要绘制曲线，就需要把迭代过程中的值保存下来，因此不能直接调用solver.solve()，需要迭代。在迭代过程中，每迭代200次测试一次：

In [ ]:
%%time
niter =4000
test_interval = 200
train_loss = np.zeros(niter)
test_acc = np.zeros(int(np.ceil(niter / test_interval)))

# the main solver loop
for it in range(niter):
    solver.step(1)  # SGD by Caffe
    
    # store the train loss
    train_loss[it] = solver.net.blobs['loss'].data
    solver.test_nets[0].forward(start='conv1')
    
    if it % test_interval == 0:
        acc=solver.test_nets[0].blobs['accuracy'].data
        print 'Iteration', it, 'testing...','accuracy:',acc
        test_acc[it // test_interval] = acc

绘制train过程中的loss曲线，和测试过程中的accuracy曲线。

In [ ]:
print(test_acc)
_, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(np.arange(niter), train_loss)
ax2.plot(test_interval * np.arange(len(test_acc)), test_acc, 'r')
ax1.set_xlabel('iteration')
ax1.set_ylabel('train loss')
ax2.set_ylabel('test accuracy')